#### Discrete p-Fuzzy Predator-Prey Model

Models ecological predator-prey interactions using discrete p-fuzzy systems where $x_{n+1} = x_n + f(x_n)$ and $f(x_n)$ is determined by fuzzy inference rules.

In [ ]:
# Install pyfuzzy-toolbox and dependencies
!pip install pyfuzzy-toolbox matplotlib numpy -q

In [ ]:
# Import libraries for fuzzy systems and visualization
import numpy as np
import matplotlib.pyplot as plt

from fuzzy_systems import MamdaniSystem
from fuzzy_systems.dynamics import PFuzzyDiscrete

print("✅ Libraries imported successfully!")

In [ ]:
# Create Mamdani system for predator-prey dynamics
fis = MamdaniSystem(name="Discrete Predator-Prey")

# ========================================
# INPUTS: Prey (x) and Predators (y)
# ========================================

# Define universe of discourse [0, 100] for both populations
fis.add_input('prey', (0, 100))
fis.add_input('predator', (0, 100))

# Linguistic terms with 4 levels: B (Low), MB (Med-Low), MA (Med-High), A (High)
for var in ['prey', 'predator']:
    fis.add_term(var, 'B', 'gaussian', (0, 12))           # Low
    fis.add_term(var, 'MB', 'gaussian', (100/3, 12))      # Med-Low
    fis.add_term(var, 'MA', 'gaussian', (2*100/3, 12))    # Med-High
    fis.add_term(var, 'A', 'gaussian', (100, 12))         # High

fis.plot_variables()

In [ ]:
# ========================================
# OUTPUTS: Population variations Δprey and Δpredator
# ========================================

# Output 1: Δprey (prey variation) - negative terms for decrease, positive for increase
var_prey_universe = (-2,2)
lrg = 0.5

fis.add_output('var_prey', var_prey_universe)
fis.add_term('var_prey', 'B_n', 'triangular', (-lrg,0,0))
fis.add_term('var_prey', 'MB_n', 'triangular', (-2*lrg,-lrg, 0))
fis.add_term('var_prey', 'MA_n', 'triangular', (-3*lrg, -2*lrg, -lrg))
fis.add_term('var_prey', 'A_n', 'trapezoidal', (-4*lrg,-4*lrg, -3*lrg, -2*lrg))
fis.add_term('var_prey', 'B_p', 'triangular', (0, 0, lrg))
fis.add_term('var_prey', 'MB_p', 'triangular', (0, lrg, 2*lrg))
fis.add_term('var_prey', 'MA_p', 'triangular', (lrg, 2*lrg, 3*lrg))
fis.add_term('var_prey', 'A_p', 'trapezoidal', (2*lrg, 3*lrg, 4*lrg,4*lrg))

# Output 2: Δpredator (predator variation) - same structure as prey variation
var_predator_universe = (-2,2)
lrg = 0.5

fis.add_output('var_predator', var_predator_universe)
fis.add_term('var_predator', 'B_n', 'triangular', (-lrg,0,0))
fis.add_term('var_predator', 'MB_n', 'triangular', (-2*lrg,-lrg, 0))
fis.add_term('var_predator', 'MA_n', 'triangular', (-3*lrg, -2*lrg, -lrg))
fis.add_term('var_predator', 'A_n', 'trapezoidal', (-4*lrg,-4*lrg, -3*lrg, -2*lrg))
fis.add_term('var_predator', 'B_p', 'triangular', (0, 0, lrg))
fis.add_term('var_predator', 'MB_p', 'triangular', (0, lrg, 2*lrg))
fis.add_term('var_predator', 'MA_p', 'triangular', (lrg, 2*lrg, 3*lrg))
fis.add_term('var_predator', 'A_p', 'trapezoidal', (2*lrg, 3*lrg, 4*lrg,4*lrg))

fis.plot_variables(['var_prey','var_predator'])


In [ ]:
# ========================================
# FUZZY RULE BASE (16 rules)
# ========================================

# Define rules encoding predator-prey interactions
# Format: ('prey_term', 'predator_term', 'var_prey_term', 'var_predator_term')

rules = [
    # Row 1: prey = B (low)
    ('B', 'B', 'MB_p', 'MB_n'),
    ('B', 'MB', 'B_p', 'MB_n'),
    ('B', 'MA', 'B_n', 'MA_n'),
    ('B', 'A', 'MB_n', 'A_n'),
    
    # Row 2: prey = MB (med-low)
    ('MB', 'B', 'MA_p', 'B_n'),
    ('MB', 'MB', 'MB_p', 'B_n'),
    ('MB', 'MA', 'B_n', 'MB_n'),
    ('MB', 'A', 'MB_n', 'MA_n'),
    
    # Row 3: prey = MA (med-high)
    ('MA', 'B', 'MB_p', 'MA_p'),
    ('MA', 'MB', 'B_p', 'MB_p'),
    ('MA', 'MA', 'MB_n', 'B_p'),
    ('MA', 'A', 'MA_n', 'B_p'),
    
    # Row 4: prey = A (high)
    ('A', 'B', 'B_n', 'A_p'),
    ('A', 'MB', 'MB_n', 'MA_p'),
    ('A', 'MA', 'MA_n', 'MB_p'),
    ('A', 'A', 'A_n', 'B_p')
]

# Add rules to the fuzzy inference system
fis.add_rules(rules)
fis.plot_rule_matrix()

# Use fis.export_rules(filename) to save the rules
# Use fis.import_rules(filename) to import rules
# Use fis.save(filename) to save the fis entirely
# Use fis.load(filename) to load a fis

In [ ]:
# Create discrete p-fuzzy dynamical system
# Mode 'absolute': x_{n+1} = x_n + f(x_n) where f is the fuzzy inference output
pfuzzy = PFuzzyDiscrete(
    fis=fis,
    mode='absolute',  # x_{n+1} = x_n + f(x_n)
    state_vars=['prey', 'predator'],
)


In [ ]:
# Simulate the predator-prey dynamics
x0 = {'prey': 50, 'predator': 40}  # Initial conditions
n_steps = 250  # Number of time steps

# Run simulation
trajectory = pfuzzy.simulate(x0=x0, n_steps=n_steps)

# Plot temporal dynamics showing oscillatory behavior
fig, ax = pfuzzy.plot_trajectory(
    variables=['prey', 'predator'],
    figsize=(12, 6),
    title='Discrete Predator-Prey System',
    xlabel='Time (steps)',
    ylabel='Population'
)

plt.show()

In [ ]:
# Visualize the phase space trajectory
# Shows the relationship between prey and predator populations over time
fig, ax = pfuzzy.plot_phase_space(
    'prey', 'predator',
    figsize=(8, 5),
    title='Phase Space: Discrete Predator-Prey'
)

plt.show()

In [ ]:
# Compare dynamics for different initial conditions
# Shows how the system behaves from various starting points
initial_conditions = [
    {'prey': 30, 'predator': 20},
    {'prey': 50, 'predator': 55},
    {'prey': 70, 'predator': 30},
    {'prey': 40, 'predator': 60}
]

colors = ['blue', 'red', 'green', 'purple']

# Create figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

for i, x0 in enumerate(initial_conditions):
    print(f"{i+1}. Prey={x0['prey']}, Predators={x0['predator']}")
    
    # Simulate
    t,traj = pfuzzy.simulate(x0=x0, n_steps=n_steps)
    time = pfuzzy.time
    
    # Plot temporal dynamics
    ax1.plot(time, traj[:, 0], '--', color=colors[i], alpha=0.5, linewidth=1.5)
    ax1.plot(time, traj[:, 1], '-', color=colors[i], linewidth=2, 
             label=f"IC{i+1}: ({x0['prey']}, {x0['predator']})")
    
    # Plot phase space
    ax2.plot(traj[:, 0], traj[:, 1], color=colors[i], linewidth=2, alpha=0.7)
    ax2.plot(traj[0, 0], traj[0, 1], 'o', color=colors[i], markersize=10)
    ax2.plot(traj[-1, 0], traj[-1, 1], 's', color=colors[i], markersize=8)

# Configure temporal plot
ax1.set_xlabel('Time (steps)', fontsize=12)
ax1.set_ylabel('Populations', fontsize=12)
ax1.set_title('Temporal Dynamics (solid line=predator, dashed=prey)', 
              fontsize=12, fontweight='bold')
ax1.legend(loc='best', fontsize=9)
ax1.grid(True, alpha=0.3)

# Configure phase space plot
ax2.set_xlabel('Prey', fontsize=12)
ax2.set_ylabel('Predator', fontsize=12)
ax2.set_title('Phase Space (o=initial, □=final)', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Export trajectory data to CSV (optional)
# Uncomment to save simulation results for further analysis
# pfuzzy.to_csv('/tmp/predator_prey_discrete.csv')

# print("✅ Data exported to: /tmp/predator_prey_discrete.csv")

# import pandas as pd
# df = pd.read_csv('/tmp/predator_prey_discrete.csv')
# print(df.head())